In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 18 16:06:22 2018

@author: Ken
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM

def normalize(df):
    newdf= df.copy()
    min_max_scaler = preprocessing.MinMaxScaler()
    for feature in df.columns:
        newdf[feature] = min_max_scaler.fit_transform(df[feature].values.reshape(-1,1))
    return newdf
    
def data_helper(df, time_frame, train_rate):
    # Feature number
    number_features = len(df.columns)
    datavalue = df.as_matrix()
    x_result = []    
    y_result = []
    for index in range(len(datavalue)-time_frame):
        x_result.append(datavalue[index: index + time_frame])
        y_result.append(datavalue[index + time_frame][:5])  # test for all value
    x_result = np.array(x_result)
    y_result = np.array(y_result)

    number_train = round((1 - train_rate) * x_result.shape[0])
    x_train, x_test = x_result[:int(number_train)], x_result[int(number_train) :]
    y_train, y_test = y_result[:int(number_train)], y_result[int(number_train) :]
    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], number_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], number_features))

    return [x_train, y_train, x_test, y_test]

def build_model(input_length, df, cell, layers, dropout):
    print(len(df.columns))
    number_features = len(df.columns)
    model = Sequential()
    for _ in range(layers):
        model.add(LSTM(cell, input_shape=(input_length, number_features), return_sequences=True))
        model.add(Dropout(dropout))
    model.add(LSTM(cell, input_shape=(input_length, number_features), return_sequences=False))
    model.add(Dropout(dropout))
    model.add(Dense(16,kernel_initializer="uniform",activation='relu'))
    model.add(Dense(5,kernel_initializer="uniform",activation='linear')) # CELL_SIZE 5, output dim
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy']) # mae
    return model

def denormalize(df, norm_value):
    denorm_value = []

    #norm_value = norm_value.T

    pre_df = pd.DataFrame(norm_value, columns = df.columns[:5])
    for feature in df.columns[:5]:
        
        original_value = df[feature].values.reshape(-1,1)
        denorm_column = pre_df[feature].values.reshape(-1,1)
    
        min_max_scaler = preprocessing.MinMaxScaler()
        min_max_scaler.fit_transform(original_value)
        denorm_value.append(min_max_scaler.inverse_transform(denorm_column))
        
    return denorm_value

def generate_model(parameters, df, csvfile):
    time_step, train_rate, cell_size, layers_num, batchsize, epochs_num, dropout_rate, activation_1, activation_2, validation_rate = parameters
    df_norm= normalize(df)
    X_train, y_train, X_test, y_test = data_helper(df_norm, time_step, train_rate)
    model = build_model(time_step, df, cell_size, layers_num, dropout_rate)
    model.fit( X_train, y_train, batch_size=batchsize, epochs=epochs_num, validation_split=validation_rate, verbose=1)
    
    pred = model.predict(X_test) # model.predict(data set)

    denorm_pred = denormalize(df, pred)
    denorm_ytest = denormalize(df, y_test)
    
    #print(model.summary()) # model summary
    #print(model.get_config()) # model configuration
    model_path = csvfile[:-4] + '_model.md' 
    model.save(model_path)
    
    #%matplotlib inline  
    for i in range(5):
        plt.plot(denorm_pred[i],color='red', label='Prediction')
        plt.plot(denorm_ytest[i],color='blue', label='Answer')
        plt.legend(loc='best')
        plt.show()
    score = model.evaluate(X_test, y_test, batch_size=batchsize)
    print(denorm_pred)
    return score[0]

def main():
    #chromosom_value=[[10,20,30],[0.1,0.2,0.3],[16,32,64,128,256],[1,2,3],[100,200,300],[100,200,300],[0.1,0.2,0.3,0.4,0.5],["sigmoid","relu","linear"],["sigmoid","relu","linear"],[0.1,0.2,0.3]]
    # 1. TIME_FRAME = [10,20,30] # input資料維度
    # 2. TRAIN_RATE = [0.1,0.2,0.3] # 切割train, test資料比例
    # 3. CELL_SIZE = ：[16,32,64,128,256] # hidden Layer中memory cells
    # 4. LAYERS_NUM = ：[1,2,3] # Layer數
    # 5. BATCH_SIZE = ：[100,200,300] # 每次批次投入資料量
    # 6. EPOCHS_NUM =  [100,200,300] # 迭代數
    # 7. DROPOUT_RATE = [0.1,0.2,0.3,0.4,0.5]3 # Dropout 比例
    # 8. activation = '' # 第一層 activation ：["sigmoid","relu","linear"],
    # 9. activation = '' # 第二層 activation ：["sigmoid","relu","linear"],
    # 10.validation rate ：[0.1,0.2,0.3]
    ''' load all csv
    csv_files =[i for i in fnmatch.filter(os.listdir('.'), '*.csv')]
    for file in csv_files:
        foxconndf= pd.read_csv(file, index_col=0 )
        foxconndf.dropna(how='any',inplace=True) # remove the rows with blank
        foxconndf = foxconndf.drop(['Date'], axis=1)
        # 調整 foxconndf columns
    ''' 

    foxconndf= pd.read_csv('0051.csv', index_col=0 )
    foxconndf.dropna(how='any',inplace=True) # remove the rows with blank
    foxconndf = foxconndf.drop(['Date'], axis=1)
    # 調整 foxconndf columns
    test = [5,0.1,50,2,None,100,0.3,'relu','linear', 0.1] # 參數list範例
    score = generate_model(test, foxconndf, '0051.csv')
    print(score)
if __name__ == '__main__':
    main()